<a href="https://colab.research.google.com/github/kuzminadya/FHP-infovis-py/blob/master/Data_Dossier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Dossier | Consultation on 11 June

## An idea

My idea is to analyse and visualise the transformation of font file changing over time. Creating a font, especially a large font family may take several years of work or it might be a result of collaboration of several designers and type engineers. Even after the first version is released, the contribution to the project and updating the font may last for several years.

*add text about version control system*

## Title (the first version)

Visualising font file transformation over time 

or

Visualisation and analysis of font file transformation over time 

or

As if the font was software. Visualization of the font development process

## An approximate work plan

[x] Choose a topic and/or dataset you are interested in\
[x] Prepare the data for the first use case\
[x] Prepare the data for the second use case\
[x] Pose a question to apply your visualization skills\
[ ] Generate several visualizations (static and interactive) → 25.06.2020\
[ ] Include the most informative in report on your process → 02.07.2020\
[ ] Share the resulting report as a Jupyter Notebook → 02.02.2020


## Use cases

There are two possible ways of elaborating my experiment. I can visualise either the structure of the existing font (the first use case) or I can create one, using UFO format (the second use case). For the second story I need to prepare a sample dataset myself.

### The first use case

As an examle I use Source Code Pro Project from Adobe, which took part for 4 years:\
https://github.com/adobe-fonts/source-code-pro/commits/release

And IBM Plex® typeface which was active on Github for a 2 years and have 219 commits:\
https://github.com/IBM/plex 

### The second use case

For the second project I decided to use my own current project [Mogee font](http://nadyakuzmina.com/story-of-mogeefont)

The font originally was made in glyphs and I need to prepare all the existing versions in UFO format.

https://robofont.com/documentation/how-tos/using-git/

> The UFO file format is perfect for version control, since it is text-based. A UFO is basically just a folder with many little XML-files in it. A UFO file could theoretically be created and modified in a text editor. This opens the field of digital type design for version control tools normally used only by software developers.

## IBM Plex®

Question

https://github.com/IBM/plex/blob/master/LICENSE.txt


In [0]:
import pandas as pd
import altair as alt

The first step is getting the dataset in JSON format from Github. It includes two files: a description of the repo and a list of commits.

https://developer.github.com/v3/repos/#get-a-repository

/repos/:owner/:repo

In [58]:
import requests 

url = ('https://api.github.com/repos/IBM/plex')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_repo_json = response.json()

# dataset_repo_json

Request returned 200 : 'OK'


https://developer.github.com/v3/repos/commits/ 

GET /repos/:owner/:repo/commits

In [59]:
url = ('https://api.github.com/repos/IBM/plex/commits')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_commits_json = response.json()

# dataset_commits_json

Request returned 200 : 'OK'


The next part is converting JSON into table.

https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

In [77]:
from pandas.io.json import json_normalize
df_commits_table = pd.DataFrame.from_dict(json_normalize(dataset_commits_json), orient='columns')
df_repo_table = pd.DataFrame.from_dict(json_normalize(dataset_repo_json), orient='columns')
df_commits_table.sample(5)
# df_repo_table

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,sha,node_id,url,html_url,comments_url,author,committer,parents,commit.author.name,commit.author.email,commit.author.date,commit.committer.name,commit.committer.email,commit.committer.date,commit.message,commit.tree.sha,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,commit.verification.payload,author.login,author.id,author.node_id,author.avatar_url,author.gravatar_id,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.id,committer.node_id,committer.avatar_url,committer.gravatar_id,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin
22,f51d07200ae54caed11da4de140142ed61af1be3,MDY6Q29tbWl0MTA1NzAyNjMxOmY1MWQwNzIwMGFlNTRjYW...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/f51d07200ae...,https://api.github.com/repos/IBM/plex/commits/...,NaN,NaN,[{'sha': '5cba621ef7cf82d813a0e794d778f4c605aa...,Vince Picone,Vincent.Patrick.Picone@ibm.com,2019-10-05T03:30:32Z,Vince Picone,Vincent.Patrick.Picone@ibm.com,2019-10-05T03:30:32Z,3.0.0,8845a8c10a253819c6e3eff3ea83d300de60c36e,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,877cd331a707864d70af9abc3541f6fd93571da3,MDY6Q29tbWl0MTA1NzAyNjMxOjg3N2NkMzMxYTcwNzg2NG...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/877cd331a70...,https://api.github.com/repos/IBM/plex/commits/...,NaN,NaN,[{'sha': '70a8b40c775df44fd9aa2d46accddc1d39df...,Vince Picone,Vincent.Patrick.Picone@ibm.com,2019-11-18T16:03:37Z,Vince Picone,Vincent.Patrick.Picone@ibm.com,2019-11-18T16:03:37Z,4.0.0,ef5182f0512791fc4d8d6d14c75314a7a75a3884,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e131c864d7cf500828cd08be657de4829880f197,MDY6Q29tbWl0MTA1NzAyNjMxOmUxMzFjODY0ZDdjZjUwMD...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/e131c864d7c...,https://api.github.com/repos/IBM/plex/commits/...,NaN,NaN,[{'sha': '612b878e01e31ec221017d973eee1df53078...,Vince Picone,Vincent.Patrick.Picone@ibm.com,2020-06-06T20:27:05Z,Vince Picone,Vincent.Patrick.Picone@ibm.com,2020-06-06T20:27:05Z,fix: tweak zip script to handle korean and var...,debd4c51481ea01f6738f79b07407d00752f8d13,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,612b878e01e31ec221017d973eee1df530785a2d,MDY6Q29tbWl0MTA1NzAyNjMxOjYxMmI4NzhlMDFlMzFlYz...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/612b878e01e...,https://api.github.com/repos/IBM/plex/commits/...,NaN,NaN,[{'sha': 'ad5cd7fedddeb12636035acb6d9bea9b500c...,Vince Picone,Vincent.Patrick.Picone@ibm.com,2020-06-01T21:31:54Z,Vince Picone,Vincent.Patrick.Picone@ibm.com,2020-06-01T21:31:54Z,fix: temporarily exclude korean from zip,aeda2c3a8100b8251b40059a832eaf53963ec6a5,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsign

In [0]:
from google.colab import files

df_commits_table.to_csv('df_commits_table.csv')
files.download('df_commits_table.csv')

In [62]:
df_repo_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 115 entries, id to organization.site_admin
dtypes: bool(11), int64(13), object(91)
memory usage: 971.0+ bytes


Datawrapper visualisation
https://datawrapper.dwcdn.net/wLs3F/1/

In [0]:
df_commits_table_reduced['commit.author.date'] = pd.to_datetime(df_commits_table_reduced['commit.author.date'], format='%Y-%m-%d')

# df_commits_table_reduced['commit.author.date']

In [64]:
alt.Chart(df_repo_table).mark_circle(size=60).encode(
    x='commit.author.date:T',
    y='',
    color='commit.author.name:N',
    tooltip=['commit.author.name:N', 'commit.message:N']
).interactive()

ValueError: ignored

alt.Chart(...)

Error:
ValueError:  encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

### Some try outs

Then hiding unnecessary columns

https://colab.research.google.com/drive/11zDWTJHup4A4rEUZq2evGVpXagElfTdT?usp=sharing

In [0]:
df_commits_table_reduced = df_commits_table.drop(columns=["sha", "node_id", "parents", "commit.author.email", "commit.committer.email", "commit.tree.sha", "commit.verification.payload",  "author.login", "author.id", "author.node_id", "author.gravatar_id", "committer.id", "committer.node_id", "committer.gravatar_id"])

# df_commits_table_reduced

In [0]:
# df_commits_table_reduced.info()

Then probably I can iterate through the dataset and collect the amount of commits that were made by each author.  

In [67]:
df_commits_groupby_name = df_commits_table_reduced.groupby('commit.author.name').count()
df_commits_groupby_name

,url,html_url,comments_url,author,committer,commit.author.date,commit.committer.name,commit.committer.date,commit.message,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,author.avatar_url,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.avatar_url,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin
commit.author.name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
LMapes,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Mike Abbink,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Vince Picone,28,28,28,0,0,28,28,28,28,28,28,28,28,28,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9


Then parsing *commit.committer.date          30 non-null     object* in datetime format


In [0]:
# df_commits_table_reduced

Couinting amount of commits for each day 

In [0]:
# df_time_index = df_commits_table_reduced.set_index("commit.author.date")
# commits_date_sums = df_time_index.resample("D").sum()
# commits_date_sums

# ha-ha, doesn't work

## Second use case

The second use case — analyse and comperison of UFO font files

xml

https://colab.research.google.com/drive/1u4gucw1Lndk_Wpuo3pAhojEUp5iiq19u

https://docs.python.org/2/library/xml.etree.elementtree.html

ufo

http://unifiedfontobject.org/versions/ufo3/

**The question:\
how to iterate through all font files on github?**


In [0]:
import xml.etree.ElementTree as ET
import requests

In order to test it I've upload each lip.plist file from all 3 versions via url to compare amount of glyphs

In [0]:
url_ver3_1 = 'https://raw.githubusercontent.com/kuzminadya/FHP-infovis-py/master/ufo/2020.04.11mogeefont-v3.1.ufo/lib.plist'
url_ver3_2 = 'https://raw.githubusercontent.com/kuzminadya/FHP-infovis-py/master/ufo/2020.04.12mogeefont-v3.2.ufo/lib.plist'
url_ver3_3 = 'https://raw.githubusercontent.com/kuzminadya/FHP-infovis-py/master/ufo/2020.04.13mogeefont-v3.3.ufo/lib.plist'
url_ver3_4 = 'https://raw.githubusercontent.com/kuzminadya/FHP-infovis-py/master/ufo/2020.04.13mogeefont-v3.4.ufo/lib.plist'

In [72]:
response_ver3 = requests.get(url_ver3_1)
ufo3_1_xml = response_ver3.text
ufo3_1_xml

'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n<dict>\n\t<key>com.schriftgestaltung.disablesAutomaticAlignment</key>\n\t<true/>\n\t<key>com.schriftgestaltung.font.customParameters</key>\n\t<array>\n\t\t<dict>\n\t\t\t<key>name</key>\n\t\t\t<string>glyphOrder</string>\n\t\t\t<key>value</key>\n\t\t\t<array>\n\t\t\t\t<string>space</string>\n\t\t\t\t<string>exclam</string>\n\t\t\t\t<string>quotesingle</string>\n\t\t\t\t<string>quotedbl</string>\n\t\t\t\t<string>numbersign</string>\n\t\t\t\t<string>dollar</string>\n\t\t\t\t<string>percent</string>\n\t\t\t\t<string>ampersand</string>\n\t\t\t\t<string>parenleft</string>\n\t\t\t\t<string>parenright</string>\n\t\t\t\t<string>asterisk</string>\n\t\t\t\t<string>plus</string>\n\t\t\t\t<string>comma</string>\n\t\t\t\t<string>hyphen</string>\n\t\t\t\t<string>period</string>\n\t\t\t\t<string>slash</string>\n\t\t\t\t<string>

In [0]:
#  print(ufo3_1_xml)

In [75]:
ufo3_1_xml_form = ET.fromstring(ufo3_1_xml)
ufo3_1_xml_form

<Element 'plist' at 0x7f1af57f81d8>

In [76]:
# I need to collect <string> inside 
# <array>
# 		<dict>
# 			<key>ascending</key>
# 			<array>
glyphs = []

for glyph in ufo3_1_xml_form.findall('.//array'): 
  glyphs.append( glyph.find('.//string').text )

# creating a dataframe 
glyphs_df = pd.DataFrame(    
    {'glyphs': glyphs,
    })  

glyphs_df

AttributeError: ignored

Error:
AttributeError                            Traceback (most recent call last)
<ipython-input-63-fa80086878b8> in <module>()
      7 
      8 for glyph in ufo3_1_xml_form.findall('.//array'):
----> 9   glyphs.append( glyph.find('.//string').text )
     10 
     11 # creating a dataframe

AttributeError: 'NoneType' object has no attribute 'text'

connect commits + visualise which particular changes has happened?\
what kind of statistic can I aggregate based on the whole font file?\
github: font file structure + commit comment // font file // each glyph separatly
